In [ ]:
!pip install SimpleITK
!pip install skan 
!pip install tifffile

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import SimpleITK as sitk
from skimage.morphology import skeletonize, thin, medial_axis
from skan import skeleton_to_csgraph, Skeleton
from skan import summarize
from skan import draw
import tifffile as tiff

import matplotlib.pyplot as plt
import numpy as np
import cv2
import plotly.express as px
import pandas as pd

In [4]:
def load_img_from_tiff(path2img):
    """
    Parameters
    ----------
    path2img: str 
        path to image Tiff file
    
    Returns
    -------
    img_array: np.array
        image data in numpy format
    """
    img = sitk.ReadImage(path2img)
    img_array = sitk.GetArrayFromImage(img)
    return(img_array)

# 2D Analysis

In [5]:
single_thresh = load_img_from_tiff("/content/drive/MyDrive/mydata/sea_urchin_data/Test n.1")
# single_thresh = cv2.resize(single_thresh, dsize=(256,256), interpolation=cv2.INTER_CUBIC)

single_thresh = single_thresh.astype('float64')
single_thresh = (single_thresh - np.min(single_thresh))/np.ptp(single_thresh)
skimage_skeleton, dist = medial_axis(single_thresh, return_distance=True)
skan_skeleton = Skeleton(dist*skimage_skeleton)
df=summarize(skan_skeleton)

### filter nodes
The branch-type is coded by number as:
0. endpoint-to-endpoint (isolated branch)
1. junction-to-endpoint
2. junction-to-junction
3. isolated cycle


In [6]:
# df.drop(df[df["branch-type"] < 2].index, inplace=True)
# df.drop(df[df["branch-type"] == 3].index, inplace=True)

In [8]:
#rename columns:
df = df.rename(columns={'image-coord-src-0': 'src-y', 
                        'image-coord-src-1': 'src-x',
                        'image-coord-dst-0': 'dst-y',
                        'image-coord-dst-1': 'dst-x',
                        'mean-pixel-value': 'thickness'})

#drop columns
df.drop([#'mean-pixel-value',
         'stdev-pixel-value',
         'coord-src-0',
         'coord-src-1',
         'coord-dst-0',
         'coord-dst-1'], axis=1, inplace=True)

In [9]:
def get_slope(x1, y1, x2, y2):
    if x1==x2:
        return ""
    m = (y2-y1)/(x2-x1)
    return m

#initiate blank columns
df["slope"] = 0
df["tortuosity"] = 0

#compute and assign values of slope and tortuosity
for index, row in df.iterrows():
    df.loc[index, 'slope']=get_slope(row["src-x"], row["src-y"], row["dst-x"], row["dst-y"])
    df.loc[index, 'tortuosity']=row["branch-distance"]/row["euclidean-distance"]

df.to_csv("single_summary.csv")

In [ ]:
df.tail()

In [ ]:
branches = pd.DataFrame(df['thickness'])
branches['source_node_id']=df['node-id-src']
branches['destination_node_id']=df['node-id-dst']
branches.to_csv("branches_filtered.csv", index=False)
branches.tail()

In [12]:
nodes_1 = pd.DataFrame(df["node-id-src"])
nodes_2=pd.DataFrame(df["node-id-dst"])

nodes_1 = nodes_1.rename(columns={'node-id-src': 'node_id'})
nodes_2 = nodes_2.rename(columns={'node-id-dst': 'node_id'})

nodes_1["node_coordinate_x"]=df['src-x']
nodes_1["node_coordinate_y"]=df['src-y']

nodes_2["node_coordinate_x"]=df['dst-x']
nodes_2["node_coordinate_y"]=df['dst-y']

final_nodes=nodes_1.append(nodes_2)

final_nodes=final_nodes.drop_duplicates(subset=['node_id',], keep='last').reset_index()
final_nodes = final_nodes.drop(['index'], axis=1)
final_nodes

final_nodes.to_csv("nodes_filtered.csv", index=False)

In [13]:
# generate_node_image (includes dst nodes)
xs_=np.array(final_nodes['node_coordinate_x'])
ys_=np.array(final_nodes['node_coordinate_y'])
node_image = np.zeros([305,305], dtype=np.uint8)
# print(node_image.shape)
node_image[ys_.astype(np.uint16), xs_.astype(np.uint16)] = 4.

In [ ]:
#compute porosity
def compute_porosity_2d(img):
    number_of_white_pix = np.sum(img == 1)  
    number_of_black_pix = np.sum(img == 0)
    porosity=number_of_black_pix/(number_of_white_pix+number_of_black_pix)
    return(porosity)

compute_porosity_2d(single_thresh)

In [ ]:
plt.rcParams["figure.figsize"] = (20,20)
f, axarr = plt.subplots(2,2)

axarr[0,0].imshow(single_thresh+skimage_skeleton, cmap='jet')
axarr[0,0].set_title("Medial Axis Skeleton")
axarr[0,1].imshow(node_image+skimage_skeleton+single_thresh, cmap='inferno')
axarr[0,1].set_title("Branch Nodes")
axarr[1,0].imshow(dist)
axarr[1,0].set_title("Distance from nearest void")
axarr[1,1].imshow(dist*skimage_skeleton+single_thresh)
axarr[1,1].set_title("Distance from nearest void (skeleton)")

In [ ]:
#branch path coords and thicknesses -> csv
branch_path_coords=[]

branch_thics=skan_skeleton.path_means()

for i in range(len(branch_thics)):
    path_arr=skan_skeleton.path_coordinates(i).tolist()
    # print(path_arr.tolist())
    #replace newlines**** w commas
    branch_path_coords.append(path_arr)
    # break

branch_path_coords_and_thickness=pd.DataFrame(branch_thics, columns=['thickness'])
branch_path_coords_and_thickness['path_coordinates']=branch_path_coords
# branch_path_coords_and_thickness['thickness']=branch_thics

branch_path_coords_and_thickness

In [34]:
branch_path_coords_and_thickness.to_csv("branch_path_coords_and_thickness.csv", index=False)

# 3D Analysis 



In [ ]:
path = "/content/drive/MyDrive/mydata/sea_urchin_data/3D/Galleria Piastra Modello 1 100x100 Echi1-10x.tif"
data = load_img_from_tiff(path)
for i in range(len(data)):
    data[i] = data[i]*(255.0/data[i].max())

In [ ]:
blurred_data = np.zeros_like(data)
for i in range(len(blurred_data)):
    blurred_data[i] = cv2.GaussianBlur(data[i], (3,3),3)

thresholded_data = np.zeros_like(data)
for i in range(len(thresholded_data)):
    bin, thresholded_data[i] = cv2.threshold(blurred_data[i],175, 255, cv2.THRESH_BINARY)

medial_axis_skel = np.zeros_like(data)
for i in range(len(thresholded_data)):
    # skel, distance = medial_axis(thresholded_data[i], return_distance=True)
    # dist_on_skel = distance * skel
    # medial_axis_skel[i]=skel
    skel = skeletonize(thresholded_data[i]//255)
    medial_axis_skel[i]=skel

In [ ]:
skan_skel = Skeleton(medial_axis_skel)
df=summarize(skan_skel)
df.to_csv("z_slices_summary.csv")
#drop brances with length < 2
# df.drop(df[df["branch-distance"] < 2].index, inplace=True)
df